In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SMU_MITB_NLP/Group Project/NLP-Lyric-Generator/src/bin

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SMU_MITB_NLP/Group Project/NLP-Lyric-Generator/src/bin


In [2]:
### Standard Imports
import numpy as np
import re
import sys
import os
from collections import Counter

import tensorflow as tf
from tensorflow.keras import layers

In [3]:
### Custom Imports
sys.path.append('../')
import lib.utilities as utils
import lib.autoencoder_utilities as ae_utils

In [4]:
### Text Parameters
start_token = '<cls>'
end_token = '<eos>'
pad_token = '<pad>'
unk_token = '<unk>'
newline_token = '<new>'
mask_token = '<mask>'

### General Parameters
random_seed = 2022
model_folder = '../../../autoencoder/lstm/v4'
model_name = 'autoencoder_lstm_with_attention_with_final_mask'

### Model Parameters
val_split = 0.2
window_len = 15
batch_size = 64
enc_dim, dec_dim = 256, 256
learn_rate = 0.001
epochs = 50
dropout = 0.05
recurrent_dropout = 0.05

In [5]:
os.makedirs(model_folder, exist_ok=True)

In [6]:
### Load Data
corpus = utils.load_corpus()
train_corpus, val_corpus, train_files, val_files = utils.split_corpus()

In [7]:
### Pre-Processing Text
_, word_count, index_to_vocab, vocab_to_index, _, _ = utils.tokenize_corpus(corpus,
                                                                            window_length = window_len,
                                                                            end_token = end_token,
                                                                            start_token = start_token,
                                                                            pad_token = pad_token,
                                                                            unk_token = unk_token,
                                                                            newline_token = newline_token,
                                                                            mask_token = mask_token)
vocab_size = len(word_count)

train_words, _, _, _, train_songs, train_songs_token_ind = utils.tokenize_corpus(train_corpus,
                                                                       window_length = window_len,
                                                                       index_to_vocab = index_to_vocab,
                                                                       vocab_to_index = vocab_to_index,
                                                                       end_token = end_token,
                                                                       start_token = start_token,
                                                                       pad_token = pad_token,
                                                                       unk_token = unk_token,
                                                                       newline_token = newline_token,
                                                                       mask_token = mask_token)

val_words, _, _, _, _, val_songs_token_ind = utils.tokenize_corpus(val_corpus,
                                                           window_length = window_len,
                                                           index_to_vocab = index_to_vocab,
                                                           vocab_to_index = vocab_to_index,
                                                           end_token = end_token,
                                                           start_token = start_token,
                                                           pad_token = pad_token,
                                                           unk_token = unk_token,
                                                           newline_token = newline_token,
                                                           mask_token = mask_token)

In [8]:
train_x_encoder, train_x_decoder, train_y = ae_utils.construct_seq_data(train_songs_token_ind, window_len)
train_x_encoder = ae_utils.mask_last(train_x_encoder, vocab_to_index, mask_token = mask_token)
val_x_encoder, val_x_decoder, val_y = ae_utils.construct_seq_data(val_songs_token_ind, window_len)
val_x_encoder = ae_utils.mask_last(val_x_encoder, vocab_to_index, mask_token = mask_token)

In [9]:
rand_int = np.random.randint(0, len(train_x_encoder), 1)[0]
print([index_to_vocab.get(x) for x in train_x_encoder[rand_int]])
print([index_to_vocab.get(x) for x in train_x_decoder[rand_int]])
print(index_to_vocab.get(train_y[rand_int]))

rand_int = np.random.randint(0, len(val_x_encoder), 1)[0]
print([index_to_vocab.get(x) for x in val_x_encoder[rand_int]])
print([index_to_vocab.get(x) for x in val_x_decoder[rand_int]])
print(index_to_vocab.get(val_y[rand_int]))

['love', 'my', 'city', 'is', 'lively', 'ways', '<new>', 'warm', 'sunny', 'days', '<new>', 'sights', 'and', 'sounds', '<mask>']
['i', 'love', 'my', 'city', 'is', 'lively', 'ways', '<new>', 'warm', 'sunny', 'days', '<new>', 'sights', 'and', 'sounds']
special
['<new>', 'i', 'have', 'queued', 'in', 'the', 'west', 'end', 'to', 'catch', 'the', 'plays', '<new>', 'discovered', '<mask>']
['taipei', '<new>', 'i', 'have', 'queued', 'in', 'the', 'west', 'end', 'to', 'catch', 'the', 'plays', '<new>', 'discovered']
i


In [10]:
train_dataset = ae_utils.construct_datasets(train_x_encoder, train_x_decoder, train_y,
                                            random_seed = random_seed,
                                            batch_size = batch_size,
                                            vocab_size = vocab_size)
val_dataset = ae_utils.construct_datasets(val_x_encoder, val_x_decoder, val_y,
                                            random_seed = random_seed,
                                            batch_size = batch_size,
                                            vocab_size = vocab_size)

In [15]:
# Encoder
encoder_input = layers.Input(shape=(window_len,vocab_size), name = 'encoder_input')

# Return state in addition to output
encoder_output, encoder_hidden_state, encoder_cell_state = layers.LSTM(enc_dim,
                                                                       dropout = dropout, recurrent_dropout = recurrent_dropout,
                                                                       return_state=True, name = "encoder_lstm")(encoder_input)

# Decoder
decoder_input = layers.Input(shape=(window_len,vocab_size), name = 'decoder_input')

# Pass the encoder state to a new LSTM, as initial state
decoder_output = layers.LSTM(dec_dim,
                             dropout = dropout, recurrent_dropout = recurrent_dropout,
                             name="decoder_lstm")(decoder_input, initial_state=[encoder_hidden_state, encoder_cell_state])

# Attention
attention_context_vector = tf.keras.layers.Attention(name = 'attention')(inputs = [decoder_output, encoder_output])

# Output
output = layers.Dense(vocab_size, name = 'output', activation = 'softmax')(tf.concat([decoder_output, attention_context_vector], 1))

model = tf.keras.Model((encoder_input, decoder_input), output, name = model_name)
model.summary()

Model: "autoencoder_lstm_with_attention_with_final_mask"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 15, 1042)]   0           []                               
                                                                                                  
 decoder_input (InputLayer)     [(None, 15, 1042)]   0           []                               
                                                                                                  
 encoder_lstm (LSTM)            [(None, 256),        1330176     ['encoder_input[0][0]']          
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                    

In [16]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate),
              metrics = ['accuracy'])

In [17]:
### Callbacks
callback_es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=10,
    verbose=1,
    mode='min',
    baseline=None,
    restore_best_weights=True
)

callback_mc = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_folder+'/weights.{epoch:02d}-{val_loss:.2f}-{val_accuracy:.2f}.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=False)

In [18]:
history = model.fit(x = train_dataset, validation_data = val_dataset, epochs = epochs, callbacks = [callback_es, callback_mc])

Epoch 1/50
180/180 [==============================] - 30s 132ms/step - loss: 5.0665 - accuracy: 0.1830 - val_loss: 5.0413 - val_accuracy: 0.1963
Epoch 2/50
180/180 [==============================] - 23s 125ms/step - loss: 4.5780 - accuracy: 0.2229 - val_loss: 4.9613 - val_accuracy: 0.1997
Epoch 3/50
180/180 [==============================] - 22s 121ms/step - loss: 4.2814 - accuracy: 0.2373 - val_loss: 4.9547 - val_accuracy: 0.2150
Epoch 4/50
180/180 [==============================] - 22s 122ms/step - loss: 3.9104 - accuracy: 0.2672 - val_loss: 4.9118 - val_accuracy: 0.2289
Epoch 5/50
180/180 [==============================] - 22s 120ms/step - loss: 3.5468 - accuracy: 0.3123 - val_loss: 4.9552 - val_accuracy: 0.2177
Epoch 6/50
180/180 [==============================] - 22s 120ms/step - loss: 3.2221 - accuracy: 0.3499 - val_loss: 5.0238 - val_accuracy: 0.2395
Epoch 7/50
180/180 [==============================] - 23s 130ms/step - loss: 2.9041 - accuracy: 0.3906 - val_loss: 5.1177 - val_ac

In [19]:
model.save_weights(f'{model_folder}/final_weights.hdf5')

In [20]:
#model.load_weights(f'{model_folder}/final_weights.hdf5')

In [21]:
prompts = ['Whenever I think back', 'And so this I know',
           'I am tired of being what you want me to be', 'Feeling so faithless, lost under the surface',
           'Relight our fire, we will find our way', 'We will rise stronger together']
result_strings = {}
results = {}
for prompt in prompts:
    result_str, result = utils.generate_text(model,
                                             ae_utils.ind_to_input_fun, ae_utils.update_input_fun,
                                             start_string = prompt,
                                             window_length = window_len,
                                             vocab_to_index_dict = vocab_to_index, index_to_vocab_dict = index_to_vocab,
                                             vocab_size = vocab_size,
                                             num_generate = 100, temperature = 1.0,
                                             random_seed = random_seed,
                                             end_token = end_token, start_token = start_token,
                                             pad_token = pad_token, unk_token = unk_token,
                                             newline_token = newline_token,
                                             depth = vocab_size)
    result_strings[prompt] = result_str
    results[prompt] = result

In [22]:
print(result_strings)

{'Whenever I think back': 'Whenever I think back \n \n a of of \n <verse> \n am my said lively struggled feel my crystals hands new uphold colleagues a not pride a the changing the in happy early \n a out to to day now her celebrations sing the childhood strength see world \n first is wide to working here country \n is island is lively aside five wind special goes tastes a joy flag special life is my lost \n favourite morning and love family \n may truly future my heart am my singapore dating though know working will family \n we first people together singapore homeland', 'And so this I know': 'And so this I know \n \n \n much for of home \n it \n we see is together \n then the hopes knew everyone now is add story \n a my strong our higher \n here a my grown home a out \n this a out my the leaving tell were is worlds lives foundations set alive you love home <verse> \n i whole are it wildest into asking remembered last my always hand ourselves our belong \n <verse> build the fair home 

In [23]:
for k, v in result_strings.items():
  with open(model_folder+f'/{model_name}-{k.lower()}.txt', 'w') as f:
      f.write(v)